In [1]:
import os
for env in ["MKL_NUM_THREADS", "NUMEXPR_NUM_THREADS", "OMP_NUM_THREADS"]:
    os.environ[env] = "1"

import numpy as np
from numpy.random import default_rng
import numpy.random as rnd
from ttictoc import tic, toc

import approxbayescomp as abc

In [2]:
import sys

print("ABC version:", abc.__version__)
print("Python version:", sys.version)
print("Numpy version:", np.__version__)

tic()

ABC version: 0.1.0
Python version: 3.8.10 | packaged by conda-forge | (default, May 11 2021, 06:25:29) 
[GCC 9.3.0]
Numpy version: 1.21.0


In [3]:
numIters = numItersData = 10
popSize = 1000
smcArgs = {"verbose": True, "numProcs": 64}

## Inference of a Poison- Frequency Dependent Exponential model

In this notebook we are are conducting a simulation experiment where the claim frequency are Negative Binomial distributed 

$$
n_s\underset{\textbf{i.i.d.}}{\sim}\text{Pois}(\lambda = 5),\text{ }s = 1,\ldots, 30
$$ 

and the individual claim sizes are freqency dependent exponential which means that 

$$
u_1,\ldots, u_{n_s}|n_s\underset{\textbf{i.i.d.}}{\sim}\text{Exp}(\beta\times e^{\delta n_s}),\text{ }s = 1,\ldots 30,
$$ 
where we set $\beta = 2$ and $\delta = 0.2$.

The available data is the total claim sizes, we have 

$$
x_s = \sum_{k = 1}^{n_s}u_k,\text{ }s = 1,\ldots, t.
$$

Our aim is to see if ABC is able to fit this model featuring dependency between claim counts and claim frequency. 

In [4]:
rg = default_rng(123)

sample_sizes = [50, 250]
T = sample_sizes[-1]
t = np.arange(1, T + 1, 1)

# Frequency-Loss Model
λ, β, δ = 4, 2, 0.2
θ_True = λ, β, δ
θ_sev = β, δ
θ_freq = λ
sev = "frequency dependent exponential"
freq = "poisson"

# Aggregation process
psi = abc.Psi("sum")

freqs, sevs = abc.simulate_claim_data(rg, T, freq, sev, θ_True)
xData = abc.compute_psi(freqs, sevs, psi)

In [5]:
[np.sum(xData[:ss] > 0) for ss in sample_sizes]

[47, 245]

In [6]:
params = ("λ", "β", "δ")
prior = abc.IndependentUniformPrior([(0, 10), (0, 20), (-1, 1)], params)
model = abc.Model("poisson", "frequency dependent exponential", psi, prior)

In [7]:
for ss in sample_sizes:
    xDataSS = xData[:ss]
    %time fit = abc.smc(numIters, popSize, xDataSS, model, **smcArgs)

Final population dists <= 2.80, ESS = 559
CPU times: user 20.5 s, sys: 1.27 s, total: 21.8 s
Wall time: 50.5 s


Final population dists <= 2.06, ESS = 799
CPU times: user 13.6 s, sys: 549 ms, total: 14.2 s
Wall time: 1min 7s


In [8]:
params = ("β", "δ")
prior = abc.IndependentUniformPrior([(0, 20), (-1, 1)], params)

In [9]:
for ss in sample_sizes:
    xDataSS = xData[:ss]
    nData = freqs[:ss]

    model = abc.Model(nData, "frequency dependent exponential", psi, prior)

    %time fit = abc.smc(numItersData, popSize, xDataSS, model, **smcArgs)

Final population dists <= 2.57, ESS = 871
CPU times: user 18.4 s, sys: 804 ms, total: 19.2 s
Wall time: 18.6 s


Final population dists <= 1.78, ESS = 916
CPU times: user 18.2 s, sys: 795 ms, total: 19 s
Wall time: 18.7 s


In [10]:
elapsed = toc()
print(f"Notebook time = {elapsed:.0f} secs = {elapsed/60:.2f} mins")

Notebook time = 156 secs = 2.61 mins
